In [8]:
import os

from pathlib import Path

import librosa
import pandas as pd
import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'librosa'

In [ ]:
data_root = "./vids1/"

dirs = os.listdir(data_root)

data_files = {
    class_: os.listdir(Path(data_root, class_)) for class_ in dirs
}

data_contents = {
    k: [] for k in data_files.keys()
}

In [ ]:
dataset = {
    "class": [],
    "filename": [],
    "chunk": [],
    "y": [],
    "sr": []
}

def split_by_seconds(dataset, root, file):
    class_ = root.split("/")[-1]
    y, sr = librosa.load(Path(root, file))
    
    n_seconds = y // sr
    for i in range(n_seconds):
        y_sample = y[i * sr : (i+1) * sr]
        dataset["class"].append(1 if class_ == "muzyka" else 0)
        dataset["filename"].append(file)
        dataset["chunk"].append(i)
        dataset["y"].append(y_sample)
        dataset["sr"].append(sr)

for root, dirs, files in os.walk(data_root):
    for file in files:
        split_by_seconds(dataset, root, file)
        
df_dataset = pd.DataFrame(dataset)

In [ ]:
def extract_mfcc(row):
    return librosa.feature.mfcc(y=row["y"], sr=row["sr"])

def extract_beat(row):
    return librosa.beat.beat_track(y=row["y"], sr=row["sr"])

def extract_rmse(row):
    return librosa.feature.rms(y=row["y"])

df_dataset["mfcc"] = df_dataset.apply(extract_mfcc, axis=1)
df_dataset["beats"] = df_dataset.apply(extract_beat, axis=1)
df_dataset["rms"] = df_dataset.apply(extract_rmse, axis=1)

In [ ]:
df_dataset

In [ ]:
df_dataset.loc[0, "mfcc"].shape

In [ ]:
X, y = np.stack(df_dataset["mfcc"]), df_dataset["class"].to_numpy()

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(20, 44)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model(X[:1]).numpy()

In [ ]:
model.fit(X, y, epochs=5)